# Sequential Plan Execution

In [1]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import RasCmdr, RasExamples, RasPrj, init_ras_project, ras

# Additional imports
import os
import numpy as np
import pandas as pd
from IPython import display
import matplotlib.pyplot as plt
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import shutil

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

📦 PIP PACKAGE MODE: Loading installed ras-commander
✓ Loaded: c:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\ras_commander\__init__.py


## Prerequisites

Before running this notebook, ensure you have:

1. **ras-commander installed**: `pip install ras-commander`
2. **Python 3.10+**: Check with `python --version`
3. **HEC-RAS 6.3+**: **REQUIRED** for plan execution
4. **Disk Space**: ~2 GB (test folder + execution results)

### What You'll Learn

This notebook demonstrates **test mode execution** using `RasCmdr.compute_test_mode()`:

- **Single Test Folder**: All plans run in one location (not separate workers)
- **Sequential Processing**: Plans execute one at a time
- **Debugging Workflow**: Easier to troubleshoot than parallel execution

### Related Notebooks

- **113_parallel_execution.ipynb** - Parallel execution for production
- **111_executing_plan_sets.ipynb** - Selective plan execution
- **110_single_plan_execution.ipynb** - Single plan fundamentals

### Key Concept: Test Mode vs Parallel Mode

**Test Mode** (`compute_test_mode`):
- Creates single `[project] [AllSequential]` folder
- Runs ONE plan at a time
- Easier debugging (one log to check)
- Slower (no parallelization)

**Parallel Mode** (`compute_parallel`):
- Creates `[project] [AllWorkers]/worker_##/` folders
- Runs MULTIPLE plans simultaneously
- Faster (if sufficient cores)
- Harder debugging (multiple logs)

**When to use Test Mode**:
1. **Debugging**: Isolate which plan is failing
2. **Limited Resources**: Only 1-2 CPU cores available
3. **Sequential Dependencies**: Plan B needs results from Plan A
4. **Development**: Testing new workflows before scaling up

**When to use Parallel Mode**:
1. **Production**: Batch processing many plans
2. **Sufficient Resources**: 4+ CPU cores available
3. **Independent Plans**: No dependencies between plans
4. **Time Critical**: Need results quickly

## Parameters

Configure these values to customize the notebook for your project.

In [2]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "Muncie"           # Example project to extract
RAS_VERSION = "6.6"               # HEC-RAS version (6.3, 6.5, 6.6, etc.)

# Execution Settings
PLAN = "01"                       # Plan number to execute
NUM_CORES = 4                     # CPU cores for 2D computation
RUN_SUFFIX = "run"                # Suffix for run folder (e.g., Muncie_run)

In [3]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from IPython import display
import matplotlib.pyplot as plt
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import shutil

In [4]:
# Extract the Balde Eagle Creek example project
from ras_commander import RasExamples

bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek", suffix="07")
print(f"Extracted project to: {bald_eagle_path}")

2025-12-28 22:37:03 - ras_commander.RasExamples - INFO - Found zip file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\Example_Projects_6_6.zip
2025-12-28 22:37:03 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2025-12-28 22:37:03 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2025-12-28 22:37:03 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-12-28 22:37:03 - ras_commander.RasExamples - INFO - Extracting project 'Balde Eagle Creek' as 'Balde Eagle Creek_07'
2025-12-28 22:37:03 - ras_commander.RasExamples - INFO - Successfully extracted project 'Balde Eagle Creek' to C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07


Extracted project to: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07


In [5]:
# define examples_dir as parent of bald_eagle_path
examples_dir = bald_eagle_path.parent
print(f"Examples directory set to: {examples_dir}")

    
# Remove any compute test folders from previous runs
for folder in examples_dir.glob("*[[]AllSequential[]]*"):
    if folder.is_dir():
        print(f"Removing existing test folder: {folder}")
        shutil.rmtree(folder)
        
for folder in examples_dir.glob("*[[]SpecificSequential*[]]*"):
    if folder.is_dir():
        print(f"Removing existing test folder: {folder}")
        shutil.rmtree(folder)


Examples directory set to: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects


## Understanding Sequential Execution in HEC-RAS

HEC-RAS simulations can be executed in several ways:

1. **Single Plan Execution**: Run one plan at a time using `RasCmdr.compute_plan()`
2. **Sequential Execution**: Run multiple plans one after another using `RasCmdr.compute_test_mode()`
3. **Parallel Execution**: Run multiple plans simultaneously using `RasCmdr.compute_parallel()`

This notebook focuses on the second approach: **Sequential Execution**. Here are the key benefits of sequential execution:

- **Controlled Resource Usage**: By running plans one at a time, you ensure consistent resource usage
- **Dependency Management**: When later plans depend on results from earlier plans
- **Simplified Debugging**: Easier to identify which plan is causing an issue when they run sequentially
- **Consistent Test Environment**: All plans run in the same isolated folder

The `compute_test_mode()` function from `RasCmdr` is specifically designed for this purpose. It creates a separate test folder, copies the project there, and executes the specified plans in sequential order.

## Downloading and Extracting Example HEC-RAS Project

Let's use the `RasExamples` class to download and extract the "Balde Eagle Creek" example project.

## Step 1: Project Initialization

Let's initialize the HEC-RAS project using the `init_ras_project()` function.

In [6]:
# Initialize the HEC-RAS project
init_ras_project(bald_eagle_path, RAS_VERSION)
print(f"Initialized HEC-RAS project: {ras.project_name}")

# Display the current plan files in the project
print("\nHEC-RAS Project Plan Data:")
display.display(ras.plan_df)

# Check how many plans we have
plan_count = len(ras.plan_df)
print(f"Found {plan_count} plans in the project")

2025-12-28 22:37:03 - ras_commander.rasmap - INFO - Successfully parsed RASMapper file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07\BaldEagle.rasmap


Initialized HEC-RAS project: BaldEagle

HEC-RAS Project Plan Data:


,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,PS Cores,DSS File,Friction Slope Method,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type
0,01,02,01,Unsteady with Bridges and Dam,5.00,UnsteadyFlow,"18FEB1999,0000,24FEB1999,0500",2MIN,1HOUR,1,...,None,dss,2,None,01,C:\Users\billk_clb\anaconda3\envs\rascmdr_pipt...,02,C:\Users\billk_clb\anaconda3\envs\rascmdr_pipt...,C:\Users\billk_clb\anaconda3\envs\rascmdr_pipt...,Unsteady
1,02,None,01,Steady Flow Run,NaN,SteadyRun,"02/18/1999,0000,02/24/1999,0500",2MIN,NaN,1,...,None,dss,1,None,01,C:\Users\billk_clb\anaconda3\envs\rascmdr_pipt...,02,C:\Users\billk_clb\anaconda3\envs\rascmdr_pipt...,C:\Users\billk_clb\anaconda3\envs\rascmdr_pipt...,Steady


Found 2 plans in the project


## Understanding the RasCmdr.compute_test_mode Method

Before we start executing plans, let's understand the `compute_test_mode()` method from the `RasCmdr` class, which we'll use for sequential execution.

### Key Parameters

- `plan_number` (str, list[str], optional): Plan number or list of plan numbers to execute. If None, all plans will be executed.
- `dest_folder_suffix` (str, optional): Suffix to append to the test folder name. Defaults to "[Test]".
- `clear_geompre` (bool, optional): Whether to clear geometry preprocessor files. Defaults to False.
- `num_cores` (int, optional): Maximum number of cores to use for each plan. If None, the current setting is not changed.
- `ras_object` (RasPrj, optional): Specific RAS object to use. If None, uses the global ras object.
- `overwrite_dest` (bool, optional): Whether to overwrite the destination folder if it exists. Defaults to False.

### Return Value
- `Dict[str, bool]`: Dictionary of plan numbers and their execution success status.

### Key Concepts

1. **Test Folder**: The function creates a separate folder with the specified suffix, copying the project there for execution.
2. **Sequential Execution**: Plans are executed one after another in the specified order.
3. **Geometry Preprocessor Files**: These files store precomputed hydraulic properties. Clearing them forces HEC-RAS to recompute these properties.
4. **Destination Folder Option**: The suffix determines the name of the test folder. Unlike `compute_plan()`, you can't specify an arbitrary destination folder.
5. **Overwrite Option**: Controls whether an existing test folder should be overwritten.

Now, let's see how this works in practice.

## Step 2: Sequential Execution of All Plans

Let's execute all plans in the project sequentially. This will create a test folder with the suffix "[AllSequential]" and run all plans one after another.

In [7]:
print("Executing all plans sequentially...")
print("This may take several minutes...")

# Record start time for performance measurement
start_time = time.time()

# Execute all plans sequentially
# - dest_folder_suffix: Suffix to append to the test folder name
# - overwrite_dest: Overwrite the destination folder if it exists
# - no ras object is specified, it will use the default "ras" object
execution_results = RasCmdr.compute_test_mode(
    dest_folder_suffix="[AllSequential]",
    overwrite_dest=True
)

# Record end time and calculate duration
end_time = time.time()
total_duration = end_time - start_time

print(f"Sequential execution of all plans completed in {total_duration:.2f} seconds")

# Create a DataFrame from the execution results for better visualization
results_df = pd.DataFrame([
    {"Plan": plan, "Success": success}
    for plan, success in execution_results.items()
])

# Display the results
print("\nExecution Results:")
display.display(results_df)

2025-12-28 22:37:03 - ras_commander.RasCmdr - INFO - Starting the compute_test_mode...
2025-12-28 22:37:03 - ras_commander.RasCmdr - INFO - Creating the test folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [AllSequential]...
2025-12-28 22:37:03 - ras_commander.RasCmdr - INFO - Copied project folder to compute folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [AllSequential]
2025-12-28 22:37:03 - ras_commander.rasmap - INFO - Successfully parsed RASMapper file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [AllSequential]\BaldEagle.rasmap
2025-12-28 22:37:03 - ras_commander.RasCmdr - INFO - Initialized RAS project in compute folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [AllSequential]\BaldEagle.pr

Executing all plans sequentially...
This may take several minutes...


2025-12-28 22:39:12 - ras_commander.RasCmdr - INFO - HEC-RAS execution completed for plan: 01
2025-12-28 22:39:12 - ras_commander.RasCmdr - INFO - Total run time for plan 01: 128.86 seconds
2025-12-28 22:39:12 - ras_commander.RasCmdr - INFO - Successfully computed plan 01
2025-12-28 22:39:12 - ras_commander.RasCmdr - INFO - Total run time for plan 01: 128.88 seconds
2025-12-28 22:39:12 - ras_commander.RasCmdr - INFO - Using ras_object with project folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [AllSequential]
2025-12-28 22:39:12 - ras_commander.RasCmdr - INFO - Running HEC-RAS from the Command Line:
2025-12-28 22:39:12 - ras_commander.RasCmdr - INFO - Running command: "C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe" -c "C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [AllSequential]\BaldEagle.prj" "C:\Users\billk_clb\anaconda3\envs\rascmdr_pipte

Sequential execution of all plans completed in 134.49 seconds

Execution Results:


,Plan,Success
0,01,True
1,02,True


### Verification: Sequential Execution

**Success Criteria**:
- Test folder created: `[project] [AllSequential]/`
- All plans executed in that folder
- HDF files created for each plan
- Plans executed one at a time (logs show sequential timing)

**Verification Code**:
```python
from pathlib import Path

# Check test folder
test_folder = bald_eagle_path.parent / "Balde Eagle Creek [AllSequential]"
assert test_folder.exists(), "Test folder not created"

# Verify each plan has results
plans_to_run = ["01", "02", "03"]  # Adjust to your plans
for plan in plans_to_run:
    hdf_files = list(test_folder.glob(f"*.p{plan}.hdf"))

    assert len(hdf_files) > 0, f"HDF not found for plan {plan}"
    assert hdf_files[0].stat().st_size > 1024, f"HDF too small for plan {plan}"

    print(f"[OK] Plan {plan}: {hdf_files[0].stat().st_size / 1e6:.1f} MB")
```

**Timing Verification**:
```python
# Sequential execution should show additive time
# Plan 01: 5 minutes
# Plan 02: 7 minutes
# Plan 03: 3 minutes
# Total: ~15 minutes (sum of individual times)

# Compare to parallel (would be ~7 minutes = max individual time)
```

**Debugging Benefits**:

With test mode, you can:
1. **Open single folder in HEC-RAS GUI**
2. **Check single log file** for all plans
3. **Step through plans** one at a time
4. **Isolate failures** more easily

Example: Find which plan is failing
```python
# Run plans one at a time in test mode
for plan in ["01", "02", "03"]:
    try:
        print(f"Testing plan {plan}...")
        RasCmdr.compute_test_mode([plan])
        print(f"  [OK] Plan {plan} succeeded")
    except Exception as e:
        print(f"  [FAILED] Plan {plan}: {e}")
        break  # Stop at first failure
```

## Step 3: Examining the Test Folder

Let's examine the test folder created by `compute_test_mode()` to better understand what happened during sequential execution.

In [8]:
# Define the test folder path
test_folder = bald_eagle_path.parent / f"Balde Eagle Creek [AllSequential]"
test_folder

WindowsPath('C:/Users/billk_clb/anaconda3/envs/rascmdr_piptest/Lib/site-packages/examples/example_projects/Balde Eagle Creek [AllSequential]')

In [9]:
# Create test folder if it doesn't exist using pathlib
if not test_folder.exists():
    test_folder.mkdir(parents=True, exist_ok=True)


if test_folder.exists():
    print(f"Test folder exists: {test_folder}")
    
    # List the key files in the test folder
    print("\nKey files in test folder:")
    
    # First, list the project file and all plan files
    prj_files = list(test_folder.glob("*.prj"))
    plan_files = list(test_folder.glob("*.p*"))
    plan_files.sort()
    
    if prj_files:
        print(f"Project file: {prj_files[0].name}")
    
    print("Plan files:")
    for plan_file in plan_files:
        file_size = plan_file.stat().st_size / 1024  # Size in KB
        print(f"  {plan_file.name}: {file_size:.1f} KB")
    
    # Look for HDF result files
    hdf_files = list(test_folder.glob("*.hdf"))
    hdf_files.sort()
    
    print("\nHDF files:")
    for hdf_file in hdf_files:
        file_size = hdf_file.stat().st_size / (1024 * 1024)  # Size in MB
        print(f"  {hdf_file.name}: {file_size:.1f} MB")
    
    # Geometry preprocessor files (if any)
    geompre_files = list(test_folder.glob("*.c*"))
    geompre_files.sort()
    
    if geompre_files:
        print("\nGeometry preprocessor files:")
        for geompre_file in geompre_files:
            file_size = geompre_file.stat().st_size / 1024  # Size in KB
            print(f"  {geompre_file.name}: {file_size:.1f} KB")
    else:
        print("\nNo geometry preprocessor files found")
        
    # Initialize a RAS project in the test folder to inspect results
    try:
        test_ras = RasPrj()
        init_ras_project(test_folder, ras.ras_exe_path, ras_object=test_ras)
        print("\nPlans with results in the test folder:")
        test_plans_with_results = test_ras.plan_df[test_ras.plan_df['HDF_Results_Path'].notna()]
        display.display(test_plans_with_results[['plan_number', 'HDF_Results_Path']])
    except Exception as e:
        print(f"Error initializing test folder as a RAS project: {e}")
else:
    print(f"Test folder not found: {test_folder}")

2025-12-28 22:39:18 - ras_commander.RasPrj - ERROR - No HEC-RAS project file found in C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek [AllSequential]


Test folder exists: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek [AllSequential]

Key files in test folder:
Plan files:

HDF files:

No geometry preprocessor files found
Error initializing test folder as a RAS project: No HEC-RAS project file found in C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek [AllSequential]. Please check the path and try again.


## Step 4: Sequential Execution of Specific Plans

Now, let's execute only specific plans in the project. We'll select plans "01" and "02" and run them sequentially with the `clear_geompre` option set to True.

In [10]:
print("Executing specific plans sequentially with clearing geometry preprocessor files...")
print("This may take several minutes...")

# Define the plans to execute
selected_plans = ["01", "02"]
print(f"Selected plans: {', '.join(selected_plans)}")

# Record start time for performance measurement
start_time = time.time()

# Execute specific plans sequentially
# - plan_number: List of plan numbers to execute
# - dest_folder_suffix: Suffix to append to the test folder name
# - clear_geompre: Clear geometry preprocessor files before execution
# - overwrite_dest: Overwrite the destination folder if it exists
execution_results = RasCmdr.compute_test_mode(
    plan_number=selected_plans,
    dest_folder_suffix="[SpecificSequentialClearGeompre]",
    clear_geompre=True,
    overwrite_dest=True
)

# Record end time and calculate duration
end_time = time.time()
total_duration = end_time - start_time

print(f"Sequential execution of specific plans completed in {total_duration:.2f} seconds")

# Create a DataFrame from the execution results for better visualization
results_df = pd.DataFrame([
    {"Plan": plan, "Success": success}
    for plan, success in execution_results.items()
])

# Display the results
print("\nExecution Results:")
display.display(results_df)

2025-12-28 22:39:18 - ras_commander.RasCmdr - INFO - Starting the compute_test_mode...
2025-12-28 22:39:18 - ras_commander.RasCmdr - INFO - Creating the test folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [SpecificSequentialClearGeompre]...
2025-12-28 22:39:18 - ras_commander.RasCmdr - INFO - Copied project folder to compute folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [SpecificSequentialClearGeompre]
2025-12-28 22:39:18 - ras_commander.rasmap - INFO - Successfully parsed RASMapper file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [SpecificSequentialClearGeompre]\BaldEagle.rasmap
2025-12-28 22:39:18 - ras_commander.RasCmdr - INFO - Initialized RAS project in compute folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_project

Executing specific plans sequentially with clearing geometry preprocessor files...
This may take several minutes...
Selected plans: 01, 02


2025-12-28 22:41:17 - ras_commander.RasCmdr - INFO - HEC-RAS execution completed for plan: 01
2025-12-28 22:41:17 - ras_commander.RasCmdr - INFO - Total run time for plan 01: 118.96 seconds
2025-12-28 22:41:17 - ras_commander.RasCmdr - INFO - Successfully computed plan 01
2025-12-28 22:41:17 - ras_commander.RasCmdr - INFO - Total run time for plan 01: 118.99 seconds
2025-12-28 22:41:17 - ras_commander.RasCmdr - INFO - Using ras_object with project folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [SpecificSequentialClearGeompre]
2025-12-28 22:41:17 - ras_commander.geom.GeomPreprocessor - INFO - Clearing geometry preprocessor file for single plan: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\Balde Eagle Creek_07 [SpecificSequentialClearGeompre]\BaldEagle.p02
2025-12-28 22:41:17 - ras_commander.geom.GeomPreprocessor - WARNING - No geometry preprocessor file found for:

Sequential execution of specific plans completed in 125.05 seconds

Execution Results:


,Plan,Success
0,01,True
1,02,True


## Sequential Execution Best Practices

### Debugging Workflow

**Step 1: Run in Test Mode First**
```python
# Before parallel execution, verify plans work
RasCmdr.compute_test_mode(["01", "02", "03"])

# If successful, then scale to parallel
RasCmdr.compute_parallel(["01", "02", "03"])
```

**Step 2: Isolate Failures**
```python
def test_plans_individually(plans):
    # Test each plan separately to find failures
    failed_plans = []

    for plan in plans:
        try:
            print(f"\nTesting plan {plan}...")
            RasCmdr.compute_test_mode([plan])
            print(f"  [OK] Plan {plan} succeeded")
        except Exception as e:
            print(f"  [FAILED] Plan {plan}: {e}")
            failed_plans.append((plan, str(e)))

    return failed_plans

# Usage
failures = test_plans_individually(["01", "02", "03", "04"])

if failures:
    print(f"\nFailed plans: {[p for p, e in failures]}")
else:
    print("\nAll plans succeeded - safe to run in parallel")
```

**Step 3: Progressive Testing**
```python
# Start with subset, then expand
subset = ["01", "02"]
RasCmdr.compute_test_mode(subset)  # Test first two

# If successful, run all
all_plans = ["01", "02", "03", "04", "05"]
RasCmdr.compute_test_mode(all_plans)
```

### Clear Geometry Preprocessor Pattern

The `clear_geompre=True` parameter is particularly useful in sequential testing:

```python
# Force geometry reprocessing for each plan
RasCmdr.compute_test_mode(
    ["01", "02", "03"],
    clear_geompre=True  # Slower but ensures clean geometry
)
```

**When to clear geometry**:
1. **Geometry modified**: Changed cross sections or mesh
2. **Troubleshooting**: Suspect geometry preprocessing issue
3. **Fresh start**: Want guaranteed clean execution

**When NOT to clear**:
1. **Geometry unchanged**: Save time by reusing preprocessed data
2. **Multiple runs**: Testing same geometry with different parameters

### LLM Forward: Test Mode Documentation

Document sequential testing results:

```python
def document_sequential_test(plans_tested, test_folder, output_file):
    import json
    from pathlib import Path
    from datetime import datetime

    test_results = {
        'test_timestamp': datetime.now().isoformat(),
        'test_folder': str(test_folder),
        'plans_tested': plans_tested,
        'results': []
    }

    for plan in plans_tested:
        hdf_files = list(test_folder.glob(f"*.p{plan}.hdf"))

        if hdf_files:
            test_results['results'].append({
                'plan_number': plan,
                'status': 'success',
                'hdf_file': str(hdf_files[0]),
                'hdf_size_mb': round(hdf_files[0].stat().st_size / 1e6, 2)
            })
        else:
            test_results['results'].append({
                'plan_number': plan,
                'status': 'failed',
                'hdf_file': None,
                'hdf_size_mb': 0
            })

    with open(output_file, 'w') as f:
        json.dump(test_results, f, indent=2)

    print(f"Test results documented: {output_file}")

# Usage
document_sequential_test(
    ["01", "02", "03"],
    test_folder,
    Path('sequential_test_results.json')
)
```

This enables:
- **Test documentation**: Record what was tested
- **Failure tracking**: Identify problematic plans
- **Regression testing**: Compare results across code changes

## Summary of Sequential Plan Execution

In this notebook, we've explored how to execute HEC-RAS plans sequentially using the RAS Commander library. Here's a summary of the key techniques we've covered:

1. **Project Setup and Initialization**: Setting up the environment and initializing a HEC-RAS project
2. **Example Project Management**: Using `RasExamples` to download and extract sample projects
3. **Basic Sequential Execution**: Using `RasCmdr.compute_test_mode()` to run all plans in a project
4. **Test Folder Analysis**: Examining the contents and results of sequential execution
5. **Selective Plan Execution**: Running specific plans with geometry preprocessor clearing

### Key Functions Used

- `init_ras_project()`: Initialize a HEC-RAS project
- `RasExamples.extract_project()`: Extract example projects for testing
- `RasCmdr.compute_test_mode()`: Run plans sequentially in a test folder
- `Path.glob()`: Examine test folder contents and results
- `RasCmdr.compute_test_mode(clear_geompre=True)`: Execute plans with preprocessor clearing

### Best Practices for Sequential Execution

1. **Environment Setup**: Ensure all required libraries are installed and properly imported
2. **Project Organization**: Clean up existing test folders before new executions
3. **Resource Management**: Monitor system resources (CPU cores, memory) for optimal performance
4. **Test Folder Naming**: Use meaningful suffixes to distinguish different execution runs
5. **Performance Tracking**: Monitor execution times for each sequential run
6. **Results Verification**: Check test folders for successful plan execution and result files
7. **Selective Execution**: Use plan filtering when only specific plans need to be run

With these techniques, you can effectively manage and execute HEC-RAS simulations sequentially, whether running all plans or a selected subset with specific configurations.